# แสดงแผนที่เส้นทาง และ สถานที่ที่น่าสนใจ มีการแนะนำสถานที่ที่น่าสนใจ และ คำอธิบายการเดินทาง

# พยายามให้แสดงสถานที่ ที่น่าสนใจตามเส้นทาง

In [ ]:
import requests
import urllib.parse
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

# Define a model for keyword extraction
class SearchKeyword(BaseModel):
    keyword: str

load_dotenv()

llm = ChatOpenAI(
    model='gpt-4o-mini',
    base_url= os.getenv("url"),  
    api_key= os.getenv("api_key"),  
    max_tokens=1000 
)

def clean_keyword(keyword: str):
    return keyword.strip().lower()

def process_user_query(user_query):
    try:
        parser = JsonOutputParser(pydantic_object=SearchKeyword)
        format_instructions = """
        คุณต้องกรองคำสำคัญจากคำขอของผู้ใช้.
        คำขอของผู้ใช้คือ: {user_query}
        คำสำคัญที่คุณต้องการหาคือสิ่งที่เกี่ยวข้องกับการค้นหาหรือการกระทำที่ผู้ใช้ต้องการ เช่น:
        - ต้องการทานข้าวหรือแวะพักทานอาหาร: "ร้านอาหาร"
        - ต้องการทานกาแฟ: "ร้านกาแฟ"
        - หากผู้ใช้ต้องการหาห้องน้ำ: "ห้องน้ำ"
        - หากผู้ใช้ต้องการเติมน้ำมัน: "ปั๊มน้ำมัน"
        - หากผู้ใช้ต้องการซื้อของฝาก: "ร้านขายของฝาก"
        - หากผู้ใช้ต้องการซื้อของ: "ร้านสะดวกซื้อ"
        - หากผู้ใช้ต้องการสถานที่แวะพักระหว่างการเดินทาง: "สถานที่พัก"
        กรุณาตอบคำสำคัญที่พบในคำขอนี้ในรูปแบบ JSON ตามตัวอย่างนี้:
        {
            "keyword": "<extracted_keyword>"
        }
        """
        
        prompt = PromptTemplate(
            template="""\
            ## คุณมีหน้าที่กรองคำสำคัญจากคำขอผู้ใช้.

            # คำขอผู้ใช้: {user_query}

            # Your response should be structured as follows:
            {format_instructions}
            """,
            input_variables=["user_query"],
            partial_variables={"format_instructions": format_instructions},
        )

        chain = prompt | llm | parser
        event = chain.invoke({"user_query": user_query})

        if event:
            keyword = event.get('keyword', '')
            cleaned_keyword = clean_keyword(keyword)
            if cleaned_keyword:
                return cleaned_keyword
    except Exception as e:
        print(f"Error processing user query: {e}")
        return None
    
def convert_locations(user_location, user_destination):
    flat, flon = user_location
    tlat, tlon = user_destination
    return flon, flat, tlon, tlat

In [ ]:
def search_logdo_map_api(keyword, user_location, radius):
    try:
        base_url = "https://api.longdo.com/mapsearch/json/search?"
        params = {
            'key': os.getenv("key_longdo"),
            'lon': user_location[1],
            'lat': user_location[0],
            'radius': radius * 1000,
            'keyword': keyword
        }
        full_url = base_url + urllib.parse.urlencode(params)

        response = requests.get(full_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Longdo API: {e}")
    return None

In [ ]:
def get_route_data(flon, flat, tlon, tlat):
    """
    Get the route data from the Longdo API based on the given start and end points.

    Args:
        flon (float): Longitude of the start point.
        flat (float): Latitude of the start point.
        tlon (float): Longitude of the destination point.
        tlat (float): Latitude of the destination point.

    Returns:
        dict: Route data returned by the Longdo API.
    """
    try:
        base_url = "https://api.longdo.com/RouteService/json/route/guide?"
        params = {
            'key': os.getenv("key_longdo"),
            'flon': flon,
            'flat': flat,
            'tlon': tlon,
            'tlat': tlat
        }
        full_url = base_url + urllib.parse.urlencode(params)

        response = requests.get(full_url)
        response.raise_for_status()
        route_data = response.json()
    
        return route_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching route data: {e}")
    return None


In [14]:
def search_places_of_interest(route_data, keyword, radius):
    """
    Search for places of interest along the route using the Longdo Map API and geocode the addresses.

    Args:
        route_data (dict): Route data containing the list of guide points.
        keyword (str): Keyword for the type of places to search for.
        radius (int): The radius to search for places (in kilometers).

    Returns:
        list: List of places of interest along the route, each with latitude and longitude.
    """
    places_of_interest = []

    for point in route_data.get('data', [])[0].get('guide', []):
        name = point.get('name')
        if name:
            try:
                address = name
                # Use Nominatim to geocode the address
                url = f'https://nominatim.openstreetmap.org/search?q={urllib.parse.quote(address)}&format=json'
                headers = {'User-Agent': 'MyGeocodingApp/1.0'}

                response = requests.get(url, headers=headers)
                response.raise_for_status()

                data = response.json()
                if data:
                    # Extract latitude and longitude of the location
                    latitude = float(data[0]["lat"])
                    longitude = float(data[0]["lon"])
                    places = search_logdo_map_api(keyword, (latitude, longitude), radius)

                    # Add the place of interest to the list
                    places_of_interest.append({
                        "place_name": name,
                        "latitude": latitude,
                        "longitude": longitude,
                        "places": places or []
                    })
                else:
                    print(f"No geocoding results for {address}.")
            except requests.exceptions.RequestException as e:
                print(f"Geocoding error for {name}: {e}")
        else:
            print("No name found in route data.")

    return places_of_interest


In [15]:
def get_places_from_route(flon, flat, tlon, tlat, keyword, radius):
    """
    Get the route and places of interest along the route.

    Args:
        flon (float): Longitude of the start point.
        flat (float): Latitude of the start point.
        tlon (float): Longitude of the destination point.
        tlat (float): Latitude of the destination point.
        keyword (str): Keyword to search for places of interest.
        radius (int): Radius in kilometers to search for places around each point.

    Returns:
        tuple: A tuple containing route data and places of interest.
    """
    route_data = get_route_data(flon, flat, tlon, tlat)

    if not route_data:
        print("Failed to fetch route data.")
        return None, []

    # Fetch places of interest along the route
    places_of_interest = search_places_of_interest(route_data, keyword, radius)

    return route_data, places_of_interest


In [16]:
if __name__ == "__main__":
    user_query = "อยากกินข้าว"
    user_location = (13.7563, 100.5018)  # กรุงเทพฯ
    user_destination = (14.022788, 99.978337)  # กาญจนบุรี
    radius = 5  # รัศมีการค้นหาสถานที่ 5 กิโลเมตร

    keyword = process_user_query(user_query)
    if keyword:
        print(f"Extracted Keyword: {keyword}")

    flon, flat, tlon, tlat = convert_locations(user_location, user_destination)

    print("\n--- Searching places along the route ---")
    route_data, places_of_interest = get_places_from_route(flon, flat, tlon, tlat, keyword, radius)

    print("\nRoute Data:")
    print(route_data)

    print("\nPlaces of Interest along the route:")
    for place in places_of_interest:
        print(place)


Extracted Keyword: ร้านอาหาร

--- Searching places along the route ---
No geocoding results for จุดกลับรถใต้สะพาน.
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.1048.

Route Data:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 99.978337, 'lat': 14.022788}, 'config': 'nullnullnullnull'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 10}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 715, 'interval': 175}, {'turn': 3, 'name': 'ถนนสมเด็จพระปิ่นเกล้า', 'distance': 173, 'interval': 29}, {'turn': 5, 'name': 'สะพานสมเด็จพระปิ่นเกล้า', 'distance': 898, 'interval': 113}, {'turn': 2, 'name': 'ถนนสมเด็จพระปิ่นเกล้า', 'distance': 256, 'interval': 21}, {'turn': 6, 'name': 'ทางคู่ขนานลอยฟ้าบรมราชชนนี', 'distance': 13501, 'interval': 655}, {'turn': 9, 'name': 'ถนนบรมราชชนนี', 'distance': 21415, 'interval': 1146}, {'turn': 2, 'name': 'ถนนเพชรเกษม', 'distance': 18633, 'interval': 1061}, {'turn': 3, 'name': '

In [17]:
import folium
from geopy.distance import geodesic
from IPython.display import display

def sort_points(points, start):
    """
    Sort the points based on distance from the starting point sequentially.

    Args:
        points (list): List of (lat, lon) tuples representing points.
        start (tuple): Starting point (lat, lon).

    Returns:
        list: List of sorted points starting from the start point.
    """
    sorted_points = [start]
    points = points.copy()
    current_point = start

    while points:
        next_point = min(points, key=lambda p: geodesic(current_point, p).meters)
        sorted_points.append(next_point)
        points.remove(next_point)
        current_point = next_point

    return sorted_points[1:]  # Exclude the starting point from the result


def show_route_with_connections(user_location, user_destination, route_points):
    """
    Display a map with a route connecting points in order and showing starting/ending points.
    
    Args:
        user_location (tuple): Latitude and longitude of the starting point (lat, lon).
        user_destination (tuple): Latitude and longitude of the destination (lat, lon).
        route_points (list): List of (lat, lon) tuples representing the sequential route.
    
    Returns:
        folium.Map: Map object to be displayed.
    """
    # Create a map centered at the midpoint between the start and end points
    midpoint = (
        (user_location[0] + user_destination[0]) / 2,
        (user_location[1] + user_destination[1]) / 2
    )
    m = folium.Map(location=midpoint, zoom_start=12)

    # Add marker for the starting point (blue)
    folium.Marker(
        location=user_location,
        popup="Start Location",
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(m)

    # Add marker for the destination (red)
    folium.Marker(
        location=user_destination,
        popup="End Location",
        icon=folium.Icon(color="red", icon="info-sign"),
    ).add_to(m)

    # Add markers for intermediate route points and draw connecting lines
    if route_points:
        # Add intermediate points
        for idx, point in enumerate(route_points):
            folium.Marker(
                location=point,
                popup=f"Point {idx + 1}",
                icon=folium.Icon(color="green", icon="info-sign"),
            ).add_to(m)

        # Draw the route as a connected line
        folium.PolyLine([user_location] + route_points + [user_destination], color="blue", weight=3, opacity=0.8).add_to(m)
    return m
    
def create_map(route_data, places_of_interest, start_location, end_location, sorted_route_points):
    """
    Create and display a map with route data and places of interest.

    Args:
        route_data (dict): Route information containing guide points.
        places_of_interest (list): List of places containing additional information.
        start_location (tuple): Starting point (lat, lon).
        end_location (tuple): Ending point (lat, lon).
        sorted_route_points (list): List of points sorted by proximity to each other.
    """
    # ตรวจสอบว่า route_data มีข้อมูลหรือไม่
    if not route_data:
        print("No route data available.")
        return

    # สร้างแผนที่เริ่มต้นโดยใช้จุดศูนย์กลางระหว่าง start และ end
    midpoint = (
        (start_location[0] + end_location[0]) / 2,
        (start_location[1] + end_location[1]) / 2
    )
    m = folium.Map(location=midpoint, zoom_start=12)

    # เพิ่ม Marker สำหรับ start และ end
    folium.Marker(
        location=start_location,
        popup="Start Location",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    folium.Marker(
        location=end_location,
        popup="End Location",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)

    # 1. ดึงข้อมูลจาก route_data และเพิ่มเส้นทาง
    route_points = sorted_route_points  # Use sorted_route_points here

    if route_points:
        # เพิ่ม Marker สำหรับจุดตามเส้นทาง
        for idx, point in enumerate(route_points):
            folium.Marker(
                location=point,
                popup=f"Point {idx + 1}",
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(m)

        # วาดเส้นทางเชื่อมจุดทั้งหมด
        folium.PolyLine([start_location] + route_points + [end_location], color="blue", weight=3, opacity=0.8).add_to(m)

    # 2. ดึงข้อมูลจาก `places_of_interest` และแสดงสถานที่ที่น่าสนใจ
    places_data = extract_and_return_data_from_places(places_of_interest)

    for data in places_data:
        place_name = data.get('place_name', 'Unknown')
        place_lat = data.get('place_lat', 'Unknown')
        place_lon = data.get('place_lon', 'Unknown')
        data_name = data.get('name', 'Unknown')
        data_lat = data.get('lat', 'Unknown')
        data_lon = data.get('lon', 'Unknown')

        # เครื่องหมายสำหรับ place_name (สีเขียว)
        if place_lat != 'Unknown' and place_lon != 'Unknown':
            folium.Marker(
                location=[place_lat, place_lon],
                popup=f"Place: {place_name}, Lat: {place_lat}, Lon: {place_lon}",
                icon=folium.Icon(color='green', icon='info-sign')
            ).add_to(m)

        # เครื่องหมายสำหรับ data_name (สีส้ม)
        if data_lat != 'Unknown' and data_lon != 'Unknown':
            folium.Marker(
                location=[data_lat, data_lon],
                popup=f"Data: {data_name}, Lat: {data_lat}, Lon: {data_lon}",
                icon=folium.Icon(color='orange', icon='info-sign')
            ).add_to(m)

    # แสดงแผนที่
    display(m)

def extract_and_return_data_from_places(places_of_interest):
    """
    Extract and return data from places_of_interest.

    Args:
        places_of_interest (list): List of places containing place_name and data.

    Returns:
        list: A list of dictionaries containing the name, latitude, and longitude of places.
    """
    extracted_data = []

    for place in places_of_interest:
        place_name = place.get('place_name', 'Unknown')
        place_lat = place.get('latitude', 'Unknown')
        place_lon = place.get('longitude', 'Unknown')

        # ตรวจสอบว่ามีข้อมูลใน 'places' หรือไม่
        places_data = place.get('places', {}).get('data', [])
        if places_data:
            for data in places_data:
                # ดึงข้อมูลที่ต้องการ
                data_name = data.get('name', 'Unknown')
                data_lat = data.get('lat', 'Unknown')
                data_lon = data.get('lon', 'Unknown')

                # เก็บข้อมูลในลิสต์
                extracted_data.append({
                    'place_name': place_name,
                    'place_lat': place_lat,
                    'place_lon': place_lon,
                    'name': data_name,
                    'lat': data_lat,
                    'lon': data_lon
                })
        else:
            # เพิ่มสถานที่หลักหากไม่มีข้อมูลย่อย
            extracted_data.append({
                'place_name': place_name,
                'place_lat': place_lat,
                'place_lon': place_lon
            })

    return extracted_data


# ตัวอย่างข้อมูลจากการเรียกใช้ API
user_query = "อยากกินข้าว"
user_location = (13.7563, 100.5018)  # จุดเริ่มต้น กรุงเทพฯ
user_destination = (14.022788, 99.978337)  # จุดสิ้นสุด กาญจนบุรี
radius = 5  # รัศมีการค้นหาสถานที่ 5 กิโลเมตร

keyword = process_user_query(user_query)  # ฟังก์ชันที่ใช้ในการแยก keyword จาก query
if keyword:
    print(f"Extracted Keyword: {keyword}")

    route_points = [
            (float(place['latitude']), float(place['longitude']))
            for place in places_of_interest
        ]

        # Sort the points based on their proximity to each other starting from the user's location
    sorted_route_points = sort_points(route_points, user_location)


# เรียกใช้ฟังก์ชันเพื่อดึงข้อมูลเส้นทางและสถานที่น่าสนใจ
flon, flat, tlon, tlat = convert_locations(user_location, user_destination)  # ฟังก์ชันที่ใช้แปลงค่าพิกัด

print("\n--- Searching places along the route ---")
route_data, places_of_interest = get_places_from_route(flon, flat, tlon, tlat, keyword, radius)

# สร้างแผนที่
create_map(route_data, places_of_interest, user_location, user_destination, sorted_route_points)


Extracted Keyword: ร้านอาหาร

--- Searching places along the route ---
No geocoding results for จุดกลับรถใต้สะพาน.
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.1048.


# ให้ LLM ทำการแนะนำสถานที่

In [18]:
def extract_and_analyze_data(places_of_interest):
    """
    Extract and analyze data from places_of_interest, considering factors like convenience, opening hours, price, and reviews.

    Args:
        places_of_interest (list): List of places containing place_name and data.

    Returns:
        list: A list of dictionaries containing place analysis including convenience, price, and reviews.
    """
    analyzed_data = []

    for place in places_of_interest:
        place_name = place.get('place_name', 'Unknown')
        place_lat = place.get('latitude', 'Unknown')
        place_lon = place.get('longitude', 'Unknown')

        # ตรวจสอบว่ามีข้อมูลใน 'places' หรือไม่
        places_data = place.get('places', {}).get('data', [])
        if places_data:
            for data in places_data:
                # ดึงข้อมูลที่ต้องการ
                data_name = data.get('name', 'Unknown')
                data_lat = data.get('lat', 'Unknown')
                data_lon = data.get('lon', 'Unknown')
                opening_hours = data.get('opening_hours', 'Unknown')  # เวลาเปิดร้าน
                price_range = data.get('price_range', 'Unknown')  # ราคา
                reviews = data.get('reviews', [])  # รายการรีวิว

                # คำนวณคะแนนจากรีวิว
                average_review_score = 0
                if reviews:
                    total_score = sum(review.get('score', 0) for review in reviews)
                    average_review_score = total_score / len(reviews)

                # คำนวณความสะดวกจากระยะทาง (การให้คะแนนตามระยะทาง)
                distance = calculate_distance(place_lat, place_lon, data_lat, data_lon)  # ฟังก์ชันคำนวณระยะทาง
                convenience_score = max(0, 10 - distance)  # คะแนนความสะดวก (ระยะทาง)

                # เก็บข้อมูลที่ได้
                analyzed_data.append({
                    'place_name': place_name,
                    'place_lat': place_lat,
                    'place_lon': place_lon,
                    'name': data_name,
                    'lat': data_lat,
                    'lon': data_lon,
                    'opening_hours': opening_hours,
                    'price_range': price_range,
                    'average_review_score': average_review_score,
                    'convenience_score': convenience_score
                })
        else:
            # เพิ่มสถานที่หลักหากไม่มีข้อมูลย่อย
            analyzed_data.append({
                'place_name': place_name,
                'place_lat': place_lat,
                'place_lon': place_lon
            })

    return analyzed_data


def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two coordinates (in kilometers).

    Args:
        lat1, lon1 (float): Coordinates of the starting point.
        lat2, lon2 (float): Coordinates of the destination point.

    Returns:
        float: The distance between the two points in kilometers.
    """
    from geopy.distance import geodesic
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

analyzed_places = extract_and_analyze_data(places_of_interest)
for place in analyzed_places:
    print(place)


{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'place_lon': 100.50187141917809, 'name': 'สระน้ำร้านอาหาร', 'lat': 15.516138172362986, 'lon': 102.69149463850029, 'opening_hours': 'Unknown', 'price_range': 'Unknown', 'average_review_score': 0, 'convenience_score': 0}
{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'place_lon': 100.50187141917809, 'name': 'สระน้ำร้านอาหาร', 'lat': 15.516061279849316, 'lon': 102.69178366997141, 'opening_hours': 'Unknown', 'price_range': 'Unknown', 'average_review_score': 0, 'convenience_score': 0}
{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'place_lon': 100.50187141917809, 'name': 'ร้านอาหารเมธาวลัย ศรแดง', 'lat': 13.7560611011446, 'lon': 100.502113871546, 'opening_hours': 'Unknown', 'price_range': 'Unknown', 'average_review_score': 0, 'convenience_score': 9.925706047790106}
{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'pl

In [19]:
def recommend_places(places_of_interest, keyword, top_n=10):
    """
    Use LLM to recommend top N places from the search results based on analyzed data.

    Args:
        places_of_interest (list): List of places containing place_name and analyzed data.
        keyword (str): The keyword used for searching.
        top_n (int): Number of top places to recommend.

    Returns:
        str: LLM response with recommendations.
    """
    if not places_of_interest:
        return "ไม่มีสถานที่ที่พบตามคำค้นหา."

    # Prepare the data to be presented to LLM, sorted by the analyzed score
    sorted_places = sorted(places_of_interest, key=lambda x: (
        x.get('average_review_score', 0) * 0.4 + 
        x.get('convenience_score', 0) * 0.3 + 
        (10 - len(x.get('price_range', 'Unknown'))) * 0.2), reverse=True)
    
    # Limit to top N places
    top_places = sorted_places[:top_n]

    places_info = "\n".join([f"{index+1}. {place['name']} (คะแนนรีวิว: {place['average_review_score']}, ความสะดวก: {place['convenience_score']}, ราคา: {place['price_range']})" for index, place in enumerate(top_places)])

    prompt = f"""
    คำค้นหาของผู้ใช้: "{keyword}"
    ต่อไปนี้คือสถานที่ที่พบจากคำค้นหา:
    {places_info}

    กรุณาแนะนำ {top_n} สถานที่ที่ดีที่สุดจากรายการนี้ โดยพิจารณาจากความน่าสนใจ, ความสะดวกในการเข้าถึง, และประเภทของสถานที่, ลักษณะการเดินทาง มีที่จอดรถหรือป่าว, รสชาติอาหารอร่อยหรือป่าว, เหมาะกับเด็กหรือผู้สูงอายุมั้ย.
    โปรดให้ข้อมูลเพิ่มเติมว่าแนะนำจากการวิเคราะห์ใด.
    """

    try:
        # Send the prompt to LLM for recommendation
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        print(f"Error generating recommendation: {e}")
        return "เกิดข้อผิดพลาดในการให้คำแนะนำ."
    

# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    places_of_interest = extract_and_analyze_data(places_of_interest)
    
    print("\n--- LLM Recommendations ---")
    recommendations = recommend_places(places_of_interest, keyword, top_n=10)
    print(recommendations)



--- LLM Recommendations ---
จากข้อมูลที่ได้รับเกี่ยวกับร้านอาหารที่ค้นหา ขอแนะนำ 10 สถานที่ที่ดีที่สุดโดยพิจารณาจากความน่าสนใจ ความสะดวกในการเข้าถึง และประเภทของสถานที่:

1. **ร้านอาหารกุ้งหลวง ซีฟู้ด** (คะแนนรีวิว: 0, ความสะดวก: 9.977741865007046)
   - **ความน่าสนใจ:** เป็นร้านซีฟู้ดที่คาดว่าจะมีเมนูหลากหลายและสดใหม่
   - **ความสะดวก:** สะดวกในการเข้าถึง
   - **การเดินทาง:** มีที่จอดรถ

2. **ร้านอาหารกุ้งหลวง ซีฟู้ด** (คะแนนรีวิว: 0, ความสะดวก: 9.934899346398852)
   - **ความน่าสนใจ:** มีชื่อเสียงในเรื่องอาหารทะเล
   - **ความสะดวก:** การเข้าถึงง่าย
   - **การเดินทาง:** มีที่จอดรถ

3. **ร้านอาหารเมธาวลัย ศรแดง** (คะแนนรีวิว: 0, ความสะดวก: 9.925706047790106)
   - **ความน่าสนใจ:** เป็นร้านที่มีเอกลักษณ์ในอาหารไทย
   - **ความสะดวก:** เดินทางสะดวก
   - **การเดินทาง:** มีที่จอดรถและเหมาะกับครอบครัว

4. **ร้านอาหารหรรษายอดผัก** (คะแนนรีวิว: 0, ความสะดวก: 9.90827145081901)
   - **ความน่าสนใจ:** เน้นการนำเสนออาหารเพื่อสุขภาพจากผัก
   - **ความสะดวก:** สะดวกสบาย
   - **การเดินทาง:** มีที่จอดรถ เ

คำแนะนำการเดินทาง

In [20]:
def describe_route(route_data):
    """
    ใช้ข้อมูลจาก API ที่ได้รับเพื่ออธิบายการเดินทางตามแต่ละจุดที่ต้องไป
    """
    steps = []
    for instruction in route_data['data'][0]['guide']:
        turn = instruction['turn']
        name = instruction['name']
        distance = instruction['distance']
        interval = instruction['interval']
        
        # สร้างคำแนะนำการเดินทางตามขั้นตอน
        step = f"เลี้ยวที่ {turn} ไปที่ {name}, ระยะทาง {distance} เมตร, ระยะห่างจากจุดที่แล้ว {interval} เมตร"
        steps.append(step)
    
    return steps

def display_route_description(steps):
    """
    แสดงคำอธิบายการเดินทาง
    """
    print("คำแนะนำการเดินทาง:")
    for step in steps:
        print(step)

if __name__ == "__main__":
    # ตัวอย่างข้อมูลจาก API

    # เรียกใช้ฟังก์ชันเพื่ออธิบายการเดินทาง
    steps = describe_route(route_data)
    display_route_description(steps)


คำแนะนำการเดินทาง:
เลี้ยวที่ 4 ไปที่ วงเวียนอนุสาวรีย์ประชาธิปไตย, ระยะทาง 39 เมตร, ระยะห่างจากจุดที่แล้ว 10 เมตร
เลี้ยวที่ 2 ไปที่ ถนนราชดำเนินกลาง, ระยะทาง 715 เมตร, ระยะห่างจากจุดที่แล้ว 175 เมตร
เลี้ยวที่ 3 ไปที่ ถนนสมเด็จพระปิ่นเกล้า, ระยะทาง 173 เมตร, ระยะห่างจากจุดที่แล้ว 29 เมตร
เลี้ยวที่ 5 ไปที่ สะพานสมเด็จพระปิ่นเกล้า, ระยะทาง 898 เมตร, ระยะห่างจากจุดที่แล้ว 150 เมตร
เลี้ยวที่ 2 ไปที่ ถนนสมเด็จพระปิ่นเกล้า, ระยะทาง 256 เมตร, ระยะห่างจากจุดที่แล้ว 22 เมตร
เลี้ยวที่ 6 ไปที่ ทางคู่ขนานลอยฟ้าบรมราชชนนี, ระยะทาง 13501 เมตร, ระยะห่างจากจุดที่แล้ว 655 เมตร
เลี้ยวที่ 9 ไปที่ ถนนบรมราชชนนี, ระยะทาง 21415 เมตร, ระยะห่างจากจุดที่แล้ว 1152 เมตร
เลี้ยวที่ 2 ไปที่ ถนนเพชรเกษม, ระยะทาง 18633 เมตร, ระยะห่างจากจุดที่แล้ว 1057 เมตร
เลี้ยวที่ 3 ไปที่ จุดกลับรถใต้สะพาน, ระยะทาง 44 เมตร, ระยะห่างจากจุดที่แล้ว 4 เมตร
เลี้ยวที่ 3 ไปที่ ถนนทรงพล, ระยะทาง 1141 เมตร, ระยะห่างจากจุดที่แล้ว 122 เมตร
เลี้ยวที่ 0 ไปที่ ทางหลวงแผ่นดินหมายเลข 321, ระยะทาง 1611 เมตร, ระยะห่างจากจุดที่แล้ว 123 เมตร
เลี้ยวที่ 

In [27]:
def explain_route_with_llm(route_data):
    route_steps = []
    for instruction in route_data['data'][0]['guide']:
        turn = instruction['turn']
        name = instruction['name']
        distance = instruction['distance']
        interval = instruction['interval']
        
        step = f"เลี้ยวที่ {turn} ไปที่ {name}, ระยะทาง {distance} เมตร, ระยะห่างจากจุดที่แล้ว {interval} เมตร"
        route_steps.append(step)

    route_description = "\n".join(route_steps)

    prompt = f"""
    นี่คือคำแนะนำการเดินทางที่ได้จาก API:
    {route_description}

    กรุณาอธิบายเส้นทางการเดินทางนี้ในรูปแบบภาษาคนที่เข้าใจง่าย
    คำอธิบายควรจะเป็นแบบการแนะนำที่เข้าใจง่ายสำหรับผู้ใช้ที่ไม่เคยเดินทางมาก่อน
    """

    try:
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        print(f"Error generating description: {e}")
        return "เกิดข้อผิดพลาดในการอธิบายการเดินทาง"
    
# ฟังก์ชันการแสดงคำอธิบายการเดินทาง
def display_route_explanation(explanation):
    print("คำอธิบายการเดินทาง:")
    print(explanation)
    
if __name__ == "__main__":
    # เรียกใช้ฟังก์ชันเพื่ออธิบายเส้นทาง
    explanation = explain_route_with_llm(route_data)
    
    # แสดงคำอธิบายการเดินทาง
    display_route_explanation(explanation)


คำอธิบายการเดินทาง:
แน่นอน! นี่คือคำแนะนำการเดินทางที่เข้าใจง่ายตามเส้นทางที่ได้จากข้อมูลที่ให้มา:

1. เริ่มต้นจากจุดที่คุณอยู่และไปยัง **วงเวียนอนุสาวรีย์ประชาธิปไตย** โดยเลี้ยวที่สี่ ระยะทางประมาณ 39 เมตร ซึ่งอยู่ห่างจากจุดที่แล้วเพียง 10 เมตรเท่านั้น

2. จากนั้นให้เลี้ยวที่สองไปที่ **ถนนราชดำเนินกลาง** ระยะทางประมาณ 715 เมตร และคุณจะใช้ระยะห่างจากจุดที่แล้วประมาณ 175 เมตร

3. ต่อไปให้เลี้ยวที่สามไปที่ **ถนนสมเด็จพระปิ่นเกล้า** ระยะทาง 173 เมตร โดยห่างจากจุดที่แล้วประมาณ 29 เมตร

4. เลี้ยวที่ห้าต่อไปยัง **สะพานสมเด็จพระปิ่นเกล้า** ระยะทาง 898 เมตร และคุณจะใช้ระยะห่างจากจุดที่แล้วประมาณ 150 เมตร

5. ให้เลี้ยวที่สองอีกครั้งไปที่ **ถนนสมเด็จพระปิ่นเกล้า** ระยะทาง 256 เมตร โดยห่างจากจุดที่แล้วประมาณ 22 เมตร

6. ต่อไปให้เลี้ยวที่หกไปยัง **ทางคู่ขนานลอยฟ้าบรมราชชนนี** ระยะทาง 13,501 เมตร และห่างจากจุดที่แล้วประมาณ 655 เมตร

7. จากนั้นให้เลี้ยวที่เก้าไปที่ **ถนนบรมราชชนนี** ระยะทาง 21,415 เมตร และห่างจากจุดที่แล้วประมาณ 1,152 เมตร

8. ต่อไปให้เลี้ยวที่สองไปที่ **ถนนเพชรเกษม** ระยะทาง 18,633